In [4]:
import nltk
import numpy as np
#import tflearn
#import tensorflow as tf
import random
import nltk.corpus
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from flask import Flask, request, Response, render_template, jsonify
import json
import string
import pickle
from os import listdir
import os
import sqlite3
import datetime
import time
from sklearn import svm

In [16]:
def dbDelete(query):
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    c.execute(query) #"DELETE FROM conversations"
    conn.commit()
    conn.close()

In [29]:
#dbCreate()
#dbSurveyCreate()
#dbInsert(0,"Hi","Hi welcome","","")
#dbDelete("DELETE FROM conversations")
#dbDelete("DROP TABLE survey")
dbSelectAll()

[(1, 'Hi', 'Hi welcome', '', '', '2019-04-27 19:38:04.293500'),
 (2,
  'jon',
  "Hi Jon, Nice to meet you. You can ask questions to know about Raja Singh's professional career.",
  '',
  '',
  '2019-04-27 19:42:39.253586'),
 (2,
  ' When will he complete his master',
  'His expected graduation date is May, 2020',
  'master',
  '',
  '2019-04-27 19:42:51.152003')]

In [30]:
dbSelectAllSurvey()

[(2,
  'jon',
  "Hi Jon, Nice to meet you. You can ask questions to know about Raja Singh's professional career.",
  'Appropriate',
  '2019-04-27 19:42:40.866898'),
 (2,
  'when will he complete his master',
  'His expected graduation date is May, 2020',
  'Appropriate',
  '2019-04-27 19:42:53.288662')]

In [26]:
def dbSelectAll():
    
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    c.execute('''SELECT * FROM conversations''')
    all_rows = c.fetchall()
    conn.commit()
    conn.close()
    return all_rows

In [12]:
conn = sqlite3.connect('resume.db')
conn.close()

In [6]:
def dbSelectAllSurvey():
    
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    c.execute('''SELECT * FROM survey''')
    all_rows = c.fetchall()
    conn.commit()
    conn.close()
    return all_rows

In [13]:
def dbCreate():
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS conversations (sessionId integer, userMessage text, botResponse text, responseContext text, previousResponseContext text, insertedDate timestamp)''')
    conn.commit()
    conn.close()

In [5]:
dbSurveyCreate()

In [14]:
def dbSurveyCreate():
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    c.execute('''CREATE TABLE IF NOT EXISTS survey (sessionId integer, userMessage text, botResponse text, rating text, insertedDate timestamp)''')
    conn.commit()
    conn.close()

In [ ]:
def dbInsertSurvey(sessionId,userMessage,botResponse,rating):
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    insertValues = (sessionId,userMessage,botResponse,rating,datetime.datetime.now())
    print(insertValues)
    print(c.execute('''INSERT INTO survey VALUES (?,?,?,?,?)''',insertValues))
    conn.commit()
    conn.close()

In [23]:
def dbInsert(sessionId,userMessage,botResponse,respContext,prevContext):
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    if sessionId == None or sessionId == '':
        c.execute('''SELECT MAX(sessionId) FROM conversations''')
        sessionId = c.fetchone()[0]+1
    elif sessionId == 0:
        sessionId = 1

    insertValues = (sessionId,userMessage,botResponse,respContext,prevContext,datetime.datetime.now())
    print(insertValues)
    c.execute('''INSERT INTO conversations VALUES (?,?,?,?,?,?)''',insertValues)
    conn.commit()
    conn.close()

In [58]:
def dbSelect(sessionId):
    session = (sessionId,)
    conn = sqlite3.connect('resume.db')
    c = conn.cursor()
    c.execute('''SELECT * FROM conversations where sessionId = ?''',session)
    all_rows = c.fetchall()
    conn.commit()
    conn.close()
    return all_rows

In [117]:
app = Flask(__name__)
vectorizer = CountVectorizer(ngram_range=(1,2),token_pattern=r'\b\w+\b',min_df=0)
tfTransformer = TfidfTransformer(use_idf = True)
#svmClassifier = svm.SVC(kernel='linear', C = 1.0,probability=True)



In [2]:
stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(string.punctuation)
stopwords.remove('not')
lemmatizer = WordNetLemmatizer()
classes = []
documents = []
intents = ''
output = []

In [ ]:
ignoreWords = ['not','i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'when', 'where', 'why', 'how', 'don', "don't", 'should', "should've", "?"]
for word in ignoreWords:
    stopwords.remove(word)

In [134]:
def loadIntents():
    
    with open('profile.json') as json_data:
        intents = json.load(json_data)
    return intents


In [150]:
def cleanText(summary):
    summary = summary.replace('*','').replace('-',' ').replace('/',' ').replace("'",' ')
    tokens_summary = [str.lower().strip(string.punctuation) for str in summary.split() if str not in stopwords]
    #tokens_summary = [str.lower().strip(string.punctuation) for str in summary.split()]
    lemma_summary = [lemmatizer.lemmatize(token) for token in tokens_summary if len(token) > 0]
    #for word in lemma_summary:
    #    wordList.append(word)
    return(' '.join(word for word in lemma_summary))
    

In [151]:

classes = []
intents = loadIntents()['intents']
for intent in intents:
    for pattern in intent['patterns']:
        documents.append((pattern, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

classes = sorted(list(set(classes)))

output_empty = [0] * len(classes)
for doc in documents:
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    output.append(output_row)


In [152]:

Y = vectorizer.fit_transform([cleanText(pattern) for pattern,tag in documents])
#Y = tfTransformer.fit_transform(X)
#svmClassifier.fit(Y, list(tag for pattern,tag in documents))


In [153]:
import tflearn
import tensorflow as tf
# reset underlying graph data
tf.reset_default_graph()
# Build neural network
net = tflearn.input_data(shape=[None,len(Y.toarray().tolist()[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation='softmax')
net = tflearn.regression(net)

# Define model and setup tensorboard
model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
# Start training (apply gradient descent algorithm)
model.fit(Y.toarray().tolist(), output, n_epoch=1000, batch_size=8)
#model.save('model.tflearn')

Training Step: 7999  | total loss: 0.39389 | time: 0.033s
| Adam | epoch: 1000 | loss: 0.39389 -- iter: 56/58
Training Step: 8000  | total loss: 0.35881 | time: 0.038s
| Adam | epoch: 1000 | loss: 0.35881 -- iter: 58/58
--


In [154]:
responseMsg = ''
tag = ''
context = ''
dataType = ''

In [185]:
def getResponse(actualMessage,previousContext):
    message = cleanText(actualMessage)
    test_Y = vectorizer.transform([message])
    #test_Y = tfTransformer.transform(test_X)
    
    ft = vectorizer.get_feature_names() 
    result = list(map(lambda row:dict(zip(ft,row)),test_Y.toarray()))
    #print(result)
    #print(list(result[0].values()))
    result_array = model.predict([list(result[0].values())]).tolist()[0]
    for item,score in zip(classes,result_array):
        print(item+': '+str(score))
    
    if max(result_array) > 0.4:
        intentTag = classes[result_array.index(max(result_array))]
    else:
        intentTag = 'unknown'
    
    
    #predicted_svm = svmClassifier.predict(test_Y)
    #intentTag = predicted_svm[0]
    
    for intent in intents:
        if intent['tag'] == intentTag:
            responseMsg = intent['responses'][0]
            #print("Response:  "+responseMsg)
            tag = intent['tag']
            print("Tag:  "+tag)
            context = intent['context']
            print("Context:  "+context)
            dataType = intent['type']
            print("Data Type:  "+dataType)
            
    
    #if tag == 'unknown' or tag == 'master' or tag == 'bachelor':
    pos_tokens = nltk.pos_tag(nltk.word_tokenize(actualMessage.lower()))
    print(pos_tokens)
    print(previousContext)
    if (intentTag == 'unknown' and previousContext == 'master') or (context == 'master'):
        for word,token in pos_tokens:
            if((token == 'WRB' and word=='where') or (word=='location')):
                responseMsg = intents[5]['specifics'][0]['location']
            elif(token=='WDT'):
                responseMsg = intents[5]['specifics'][0]['university']
            elif((token == 'WRB' and word=='when') or (word == 'year')) :
                responseMsg = intents[5]['specifics'][0]['year'] 
            elif(word == 'gpa'):
                responseMsg = intents[5]['specifics'][0]['gpa'] 
            elif(word == 'major'):
                responseMsg = intents[5]['specifics'][0]['major'] 
        tag = previousContext
    if (intentTag == 'unknown' and previousContext == 'bachelor') or (context == 'bachelor'):
        for word,token in pos_tokens:
            if((token == 'WRB' and word=='where') or (word=='location')):
                responseMsg = intents[4]['specifics'][0]['location']
            elif(token=='WDT'):
                responseMsg = intents[4]['specifics'][0]['university']
            elif((token == 'WRB' and word=='when') or (word == 'year')) :
                responseMsg = intents[4]['specifics'][0]['year'] 
            elif(word == 'gpa'):
                responseMsg = intents[4]['specifics'][0]['gpa'] 
            elif(word == 'major'):
                responseMsg = intents[4]['specifics'][0]['major'] 
        tag = previousContext
            
    print("Response:  "+responseMsg)
    return (responseMsg,tag)

In [188]:
message = "Tell about his bachelor's education"
response, previousContext = getResponse(message,previousContext)
print(previousContext)

bachelor: 0.9866346120834351
education: 0.00383388833142817
goodbye: 1.243123620042752e-06
greeting: 3.006251176884689e-07
master: 0.004805710632354021
thanks: 0.0044678617268800735
unknown: 0.00025631138123571873
Tag:  bachelor
Context:  bachelor
Data Type:  data
[('tell', 'NN'), ('about', 'IN'), ('his', 'PRP$'), ('bachelor', 'NN'), ("'s", 'POS'), ('education', 'NN')]
master
Response:  Bachelor's Degree - Saranathan College of Engineering, Trichy, India
master


In [167]:
nltk.pos_tag(nltk.word_tokenize(message.lower()))

[('how', 'WRB'), ('are', 'VBP'), ('you', 'PRP')]

In [162]:
actualMessage = "do not you know about his master degree"
words = actualMessage.split(' ')
pos_tokens = nltk.pos_tag(nltk.word_tokenize(actualMessage.lower()))
pos_tokens
[word for word,tag in nltk.pos_tag(nltk.word_tokenize(actualMessage.lower())) if tag not in ['PRP','IN','PRP$','TO','UH']]

['do', 'not', 'know', 'master', 'degree']

In [131]:
lemma_summary = [lemmatizer.lemmatize(token) for token in actualMessage.split() if len(token) > 0]
lemma_summary

['in', 'which', 'university', 'did', 'he', 'do', 'his', 'master']

In [167]:
[word for word,tag in pos_tokens  if 'which' in word and 'he' in word]

[]

In [145]:
pos_tokens

[('in', 'IN'),
 ('which', 'WDT'),
 ('university', 'NN'),
 ('did', 'VBD'),
 ('he', 'PRP'),
 ('do', 'VB'),
 ('his', 'PRP$'),
 ('master', 'NN')]

In [168]:
intents

[{'tag': 'greeting',
  'patterns': ['Hi',
   'How are you',
   'Is anyone there?',
   'Hello',
   'Good day',
   'Hey, How are you doing?'],
  'responses': ['Hello, thanks for visiting',
   'Good to see you',
   'Hi there, how can I help?'],
  'context': '',
  'type': ''},
 {'tag': 'goodbye',
  'patterns': ['Bye', 'See you later', 'Goodbye'],
  'responses': ['See you later, thanks for visiting',
   'Have a nice day',
   'Bye! Come back again soon.'],
  'context': '',
  'type': ''},
 {'tag': 'thanks',
  'patterns': ['Thanks', 'Thank you', "That's helpful"],
  'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
  'context': '',
  'type': ''},
 {'tag': 'education',
  'patterns': ['Tell about your educational qualifications',
   'education details of him',
   'Tell about his education'],
  'responses': ["Master's Degree - University of South Florida, Tampa, Florida. \nBachelor's Degree - Saranathan College of Engineering, Trichy, India"],
  'context': 'education',
  'type': 'list'

In [178]:
#intents = loadIntents()['intents']
intents[7]

{'tag': 'skills',
 'patterns': ['what are your core skills',
  'what are your primary skills',
  'what is your primary skillset'],
 'responses': ['My primary skills are Python, Big Data, Tableau, Machine Learning'],
 'specifics': [{'reporting': ['I have experience on reporting tooks like Tableau, as a part of my curriculum and have done project as well'],
   'big data': ['I have knowledge and experience on working in Hadoop, PySpark and Hive'],
   'machine learning': ['I have the knowledge of building machine learning models like Neural networks, SVM, Ensembles']}],
 'context': 'skills',
 'type': 'data'}

In [14]:
from pattern.en import Spelling

word = "amazzziiing"

correct_word = Spelling(word) 
print(correct_word)

IndexError: list index out of range